### Importacoes

In [1]:
# Imports

# Matemaricos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados e arquivos
import re
import requests
from pathlib import Path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Carregamento de dados, limpeza de colunas e linhas vazias e adequacao do tipo de dado

In [2]:
# Carregamento da listagem de propostas
listagem = pd.DataFrame(pd.read_excel('../Listagem/listagem_17.xlsx'))

In [3]:
# Limpeza de colunas vazias
listagem = listagem.drop(listagem.columns[[1,3,5,7,9]], axis=1)

In [4]:
# Limpeza de linhas varias (NaN)
listagem = listagem.dropna()

In [5]:
listagem['Projeto'] = listagem['Projeto'].astype(int)
listagem['Ano'] = listagem['Ano'].astype(int)

### Criacao e preenchimento de novas colunas

In [6]:
'''Criacao das colunas: 
codigo do tipo
id do projeto
data de proposicao do projeto
status da tramitacao
conferência da situacao
'''
listagem['Codigo_Tipo'] = ''
listagem['Id_Projeto'] = ''
listagem['Data_Projeto'] = ''
listagem['Tramitacao_Encerrada'] = ''
listagem['Aprovado'] = '' 

#listagem['Tramitacao'] = '' # UNIAO DOS DOIS CAMPOS ANTERIORES EM REVISAO

In [7]:
# Preenchimento do campo Codigo_Tipo
unknown_count = 0

for i in listagem.index:
    #print(listagem["Tipo"][i])
    if(listagem["Tipo"][i] == "Requerimento"):
        listagem['Codigo_Tipo'][i] = 'REQ'
    elif(listagem["Tipo"][i] == "Projeto de Lei"):
        listagem['Codigo_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Lei Complementar"):
        listagem['Codigo_Tipo'][i] = 'PLEIC'
    elif(listagem["Tipo"][i] == "Representação"):
        listagem['Codigo_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Resolução"):
        listagem['Codigo_Tipo'][i] = 'PRES'
    elif(listagem["Tipo"][i] == "Pedido de Informação"):
        listagem['Codigo_Tipo'][i] = 'PEDIF'
    elif(listagem["Tipo"][i] == "Moção"):
        listagem['Codigo_Tipo'][i] = 'MOC'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei)"):
        listagem['Codigo_Tipo'][i] = 'MSGPL'
    elif(listagem["Tipo"][i] == "Indicação"):
        listagem['Codigo_Tipo'][i] = 'IND'
    elif(listagem["Tipo"][i] == "Projeto de Decreto Legislativo"):
        listagem['Codigo_Tipo'][i] = 'PDL'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo"):
        listagem['Codigo_Tipo'][i] = 'MSGEX'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei Complementar)"):
        listagem['Codigo_Tipo'][i] = 'MSGPC'
    elif(listagem["Tipo"][i] == "Prestação de Contas"):
        listagem['Codigo_Tipo'][i] = 'PTCTS'
    elif(listagem["Tipo"][i] == "Projeto de Emenda à Lei Orgânica"):
        listagem['Codigo_Tipo'][i] = 'PELOR'
    elif(listagem["Tipo"][i] == "Projeto de Lei de Iniciativa Popular"):
        listagem['Codigo_Tipo'][i] = 'PLEIP'
    elif(listagem["Tipo"][i] == "Projeto de Resolução - Comissão de Exame Denúncia"):
        listagem['Codigo_Tipo'][i] = 'PRCED'
    elif(listagem["Tipo"][i] == "Proposta de Revisão da Lei Orgânica Municipal"):
        listagem['Codigo_Tipo'][i] = 'PRLOM'
    elif(listagem["Tipo"][i] == "Proposta de Revisão do Regimento Interno"):
        listagem['Codigo_Tipo'][i] = 'PRRI'
    elif(listagem["Tipo"][i] == "Requerimento Popular"):
        listagem['Codigo_Tipo'][i] = 'RPO'
    elif(listagem["Tipo"][i] == "Requerimento de CPI"):
        listagem['Codigo_Tipo'][i] = 'RCPI'
    elif(listagem["Tipo"][i] == "Pedido de Informação FiscalizaJF"):
        listagem['Codigo_Tipo'][i] = 'PEDFF'
    else:
        listagem['Codigo_Tipo'][i] = 'DESCONHECIDO'
        unknown_count += 1

print(f'Fim do processo. Traducao realizada com {unknown_count} registros nao encontrados')

Fim do processo. Traducao realizada com 0 registros nao encontrados


### Importacoes

In [26]:
# Preenchimento do campo Id_Projeto
count = 1
failed_req= 0
failed = False
id_projeto = ''
max_failed_req = 5

while count > 0:
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '': 
            id_projeto = ''
            failed = False
            # Variaveis de requisicao
            num_proj = int(listagem['Projeto'][i])
            ano_proj = int(listagem['Ano'][i])
            tipo_proj = listagem['Codigo_Tipo'][i]

            try:
                # Requisicao para o portal de propostas da Camara
                query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                except:
                    try:
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        id_projeto = re.search('PROPOSICAON.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                    except:
                        id_projeto = 'Nao Encontrado'
            except:
                #Assinala que houve falha
                failed = True
                    
            listagem['Id_Projeto'][i] = id_projeto
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
            resposta.close()

        time.sleep(1)
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break

    if failed_req > max_failed_req:
        break   
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print('O endpoint nao esta respondendo, verifique a conexao. Fim do processo')
else:
    print('Fim do processo')

Fim do processo


In [19]:
# Preenchimento do campo Data_Projeto
count = 1
failed_req= 0
failed = False
data_projeto = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '': 
            data_projeto = ''
            failed = False
            # Variaveis de requisicao
            id_proj = listagem['Id_Projeto'][i]
            
            try:
                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                                
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                except:
                    data_projeto = 'Data nao encontrada'
                    notfound_count += 1
            except:
                failed = True

            listagem['Data_Projeto'][i] = data_projeto
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
            resposta.close()

        time.sleep(1)
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print('O endpoint nao esta respondendo, verifique a conexao. Fim do processo')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')

Fim do processo. Nao encontrados = 25


In [20]:
# Preenchimento do campo Tramitacao_Encerrada
count = 1
failed_req= 0
failed = False
tramitacao_encerrada = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Tramitacao_Encerrada'][i] == '': 
            tramitacao_encerrada = ''
            failed = False
            
            try:
                # Variaveis de requisicao
                id_proj = listagem['Id_Projeto'][i]

                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                #query = {"numero": 44920}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)

                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                    if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                        tramitacao_encerrada = 'SIM'
                    else:
                        tramitacao_encerrada = 'NAO'
                except:
                    tramitacao_encerrada = 'Informacao nao encontrada'
                    notfound_count+=1
            except:
                failed = True

            listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
            resposta.close()

        time.sleep(1)
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Tramitacao_Encerrada'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print('O endpoint nao esta respondendo, verifique a conexao. Fim do processo')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')

O endpoint nao esta respondendo, verifique a conexao. Fim do processo


In [ ]:
# Preenchimento do campo Aprovado
count = 1
failed_req= 0
failed = False
aprovado = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Aprovado'][i] == '':
            aprovado = ''
            if listagem['Tramitacao_Encerrada'][i] == 'NAO': 
                aprovado = 'TRAMITACAO'

            elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                if listagem['Situacao'][i] != 'Arquivada' or 'Vetado Totalmente' in listagem['Situacao'][i] :
                    aprovado = 'SIM'
                else:
                    aprovado = 'NAO'                

            elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                failed = False

                try:
                    # Variaveis de requisicao
                    id_proj = listagem['Id_Projeto'][i]

                    # Requisicao para o portal de propostas da Camara
                    query = {"num": id_proj}
                    #query = {"numero": 44920}
                    resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)

                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                        if(re.search('APROVAD', retorno)):
                            aprovado = 'SIM'
                        else:
                            aprovado = 'NAO'
                    except:
                        aprovado = 'Informacao nao encontrada'
                        notfound_count += 1
                except:
                    failed = True

            listagem['Aprovado'][i] = aprovado
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
            resposta.close()

        time.sleep(1)
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Aprovado'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print('O endpoint nao esta respondendo, verifique a conexao. Fim do processo')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')

### Exportando informacoes sobre projetos e autoria para arquivos CSV

In [26]:
# Exportacao da listagem de projetos para arquivo csv
path_to_file = '../ListagemExp/ListagemExport17_Incompleta_10-04.csv'
path = Path(path_to_file)

exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situação', 'Codigo_Tipo', 
               'Id_Projeto', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

In [ ]:
# Logica de identificador de autor (Id_Autor) e identificacao de autoria, separando casos de 'multiautoria'
autoria_df_estrutura = {'nome_autor': [''],'id_projeto': ['']}
autoria_df = pd.DataFrame(columns=['nome_autor','id_projeto'])
autores = ['']
aut = ['']

for i in listagem.index:
    aut = listagem['Autor'][i].split(', ')
    
    
    for item in aut:
        autoria = pd.DataFrame({'nome_autor': item, 
                      'id_projeto': listagem['Id_Projeto'][i]}, index=[0]) 
        autoria_df = autoria_df.append(autoria, ignore_index=True, verify_integrity=False, sort=None) 

# Exportacao da listagem de autorias para arquivo csv
path_to_file = '../ListagemExp/AutoriasExport.csv'
path = Path(path_to_file)
exp_columns = ["id_projeto", "nome_autor"]

if path.is_file():
    exp_header = False
else:    
    exp_header = True

autoria_df.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

# Visualizacao do dataframe e testes

In [5]:
#Reimportando arquivo - caso execucao tenha sido pausada
path_to_file = '../ListagemExp/ListagemExport_17Incompleta2.csv'
listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situação','Codigo_Tipo'
                                 ,'Id_Projeto','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Id_Projeto'] = listagem['Id_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

In [24]:
display(listagem)

,Projeto,Ano,Tipo,Ementa,Situação,Codigo_Tipo,Id_Projeto,Data_Projeto,Tramitacao_Encerrada,Aprovado
0,2269,2017,Requerimento,limpeza do bueiro localizado na Rua Carolina d...,Arquivada,REQ,60890,22/05/2017,SIM,
1,5353,2017,Requerimento,Tapa buraco - Rua Vicente Soares Silva; Bairro...,Aprovado,REQ,64735,14/12/2017,SIM,
2,5352,2017,Requerimento,Tapa Buraco - Travessa Santo Expedito; Bairro ...,Aprovado,REQ,64734,14/12/2017,SIM,
3,5351,2017,Requerimento,Tapa Buraco - Rua Sargento Cunha; Bairro Bande...,Aprovado,REQ,64733,14/12/2017,SIM,
4,5350,2017,Requerimento,Tapa buraco - Rua Senhor Mozzato; Bairro Recan...,Aprovado,REQ,64732,14/12/2017,SIM,
...,...,...,...,...,...,...,...,...,...,...
6381,1,2017,Projeto de Lei Complementar,"""Altera o rol de atividades permitidas em Zona...",Transformado em Norma Jurídica,PLEIC,58226,04/01/2017,,
6382,1,2017,Projeto de Resolução,Altera dispositivos do Regimento Interno da Câ...,Transformado em Norma Jurídica,PRES,58224,02/01/2017,,
6383,1,2017,Projeto de Emenda à Lei Orgânica,Acrescenta o Parágrafo Único ao Art. 85 da Lei...,Transformado em Norma Jurídica,PELOR,59954,19/04/2017,,
6384,1,2017,Pedido de Informação FiscalizaJF,Solicita informações referentes a praça Adalbe...,Arquivada,PEDFF,59295,14/02/2017,,


In [25]:

count=0
count2=0
for i in listagem.index:
    if listagem['Tramitacao_Encerrada'][i]!='':
        count+=1
    else:
        count2+=1
print(count)
print(count2)
        
'''
  
    
# Contagem e conferencia de qual eh o ultimo projeto com id_projeto preenchido 
count = 0
for i in listagem.index:
    
    if listagem['Id_Projeto'][i] == "":
        print(listagem['Projeto'][i], listagem['Codigo_Tipo'][i], listagem['Ano'][i], listagem['Id_Projeto'][i])
        print(count)
        break
    else:
        count +=1
'''

2333
4053


'\n  \n    \n# Contagem e conferencia de qual eh o ultimo projeto com id_projeto preenchido \ncount = 0\nfor i in listagem.index:\n    \n    if listagem[\'Id_Projeto\'][i] == "":\n        print(listagem[\'Projeto\'][i], listagem[\'Codigo_Tipo\'][i], listagem[\'Ano\'][i], listagem[\'Id_Projeto\'][i])\n        print(count)\n        break\n    else:\n        count +=1\n'

In [ ]:
'''
# Conferencia de valores
i = 936

print(listagem['Projeto'][i], listagem['Codigo_Tipo'][i], listagem['Ano'][i], 
      listagem['Id_Projeto'][i], listagem['Data_Projeto'][i], listagem['Situação'][i])

#https://www.camarajf.mg.gov.br/sal/proposicao.php?num=45773
'''

In [ ]:
'''
# Exportacao da listagem de projetos para arquivo csv
path_to_file = '../ListagemExp/ListagemExport.csv'
path = Path(path_to_file)

#exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situação', 'Codigo_Tipo', 
#               'Id_Projeto', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header) 
'''